# ``jenga`` Corruptions

This notebook illustrates some of the corruptions used in jenga

Most corruptions are operating on tabular data, for which different sampling mechanisms are used. 

Inspired by research on missing values, we consider three different sampling schemes for corrupting tabular data (we'll keep using the 'missingness' terminology here, even if we use other corruptions than discarding data):

* **MCAR** Missing completely at random
* **MAR** Missing at random - corruption is conditioned on other column
* **MNAR** Missing not at random - corruption is conditioned on values in column on which it is applied


In [1]:
%load_ext autoreload
%autoreload 2

import random
import pandas as pd
import numpy as np

from jenga.corruptions.generic import MissingValues

In [2]:
def new_df(N=20):
    return pd.DataFrame({
        'A':np.arange(0,N) / N, 
        'B': [random.choice(['a','b','c']) for _ in range(N)],
        'C': [random.choice(['foo','bah']) for _ in range(N)]
        })

## Missing Values

Below some examples on different row sampling strategies for corruptions with missing values

In [3]:
from jenga.corruptions.generic import MissingValues

df = new_df()

df['B_MCAR'] = MissingValues(column='B', fraction=.5, missingness='MCAR').transform(df)['B']
df['B_MAR'] = MissingValues(column='B', fraction=.5, missingness='MAR').transform(df)['B']
df['B_MNAR'] = MissingValues(column='B', fraction=.5, missingness='MNAR').transform(df)['B']

df

,A,B,C,B_MCAR,B_MAR,B_MNAR
0,0.00,a,foo,NaN,a,a
1,0.05,b,foo,b,b,NaN
2,0.10,b,foo,b,NaN,NaN
3,0.15,c,bah,c,c,c
4,0.20,a,bah,NaN,a,a
5,0.25,c,bah,NaN,NaN,c
6,0.30,c,foo,NaN,NaN,c
7,0.35,b,foo,b,NaN,NaN
8,0.40,b,foo,b,NaN,NaN
9,0.45,c,bah,NaN,c,c


If we sort according to the values in the column on which the corruption as applied, we see that the 'MNAR' condition works as it should

In [4]:
df.sort_values('B')

,A,B,C,B_MCAR,B_MAR,B_MNAR
0,0.00,a,foo,NaN,a,a
4,0.20,a,bah,NaN,a,a
15,0.75,a,foo,NaN,NaN,a
14,0.70,a,foo,a,NaN,NaN
11,0.55,a,foo,NaN,NaN,NaN
12,0.60,a,bah,NaN,a,NaN
1,0.05,b,foo,b,b,NaN
2,0.10,b,foo,b,NaN,NaN
17,0.85,b,bah,b,b,NaN
7,0.35,b,foo,b,NaN,NaN


## Swapping Values

Also works between numeric and non-numeric values, with different sampling schemes

In [5]:
from jenga.corruptions.generic import SwappedValues
df = new_df()
df['C_swapped_MAR'] = SwappedValues(column='B', fraction=.5, sampling='MAR').transform(df)['C']
df['C_swapped_MCAR'] = SwappedValues(column='B', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_swapped_MCAR'] = SwappedValues(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,C_swapped_MAR,C_swapped_MCAR,A_swapped_MCAR
0,0.00,c,foo,foo,foo,foo
1,0.05,a,bah,bah,bah,bah
2,0.10,b,bah,bah,bah,0.1
3,0.15,b,bah,bah,bah,bah
4,0.20,b,bah,bah,bah,bah
5,0.25,c,bah,bah,bah,0.25
6,0.30,a,bah,bah,bah,0.3
7,0.35,a,foo,foo,foo,0.35
8,0.40,a,bah,bah,bah,bah
9,0.45,a,foo,foo,foo,0.45


## Messing up categorical Values

These corruptions permute the histogram of values

In [6]:
from jenga.corruptions.generic import CategoricalShift
df = new_df()
df['C_permuted_MAR'] = CategoricalShift(column='B', fraction=.5, sampling='MAR').transform(df)['C']
df['C_permuted_MCAR'] = CategoricalShift(column='B', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_permuted_MCAR'] = CategoricalShift(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

CategoricalShift implemented only for categorical variables


,A,B,C,C_permuted_MAR,C_permuted_MCAR,A_permuted_MCAR
0,0.00,b,foo,foo,foo,0.00
1,0.05,b,bah,bah,bah,0.05
2,0.10,a,bah,bah,bah,0.10
3,0.15,a,bah,bah,bah,0.15
4,0.20,b,bah,bah,bah,0.20
5,0.25,b,foo,foo,foo,0.25
6,0.30,c,foo,foo,foo,0.30
7,0.35,b,foo,foo,foo,0.35
8,0.40,c,foo,foo,foo,0.40
9,0.45,c,foo,foo,foo,0.45


## Adding Gaussian Noise



In [7]:
from jenga.corruptions.numerical import GaussianNoise
df = new_df()
df['A_GaussNoise_MAR'] = GaussianNoise(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_GaussNoise__MCAR'] = GaussianNoise(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,A_GaussNoise_MAR,A_GaussNoise__MCAR
0,0.00,a,bah,0.000000,0.474711
1,0.05,a,foo,0.050000,0.450812
2,0.10,a,foo,0.100000,0.100000
3,0.15,c,bah,0.619786,0.421035
4,0.20,c,bah,-0.318380,2.235660
5,0.25,c,foo,-0.032971,1.927039
6,0.30,c,foo,0.300000,-0.808729
7,0.35,c,foo,-0.196570,0.350000
8,0.40,c,bah,1.303015,0.400000
9,0.45,b,foo,0.450000,-0.236334


## Scaling numeric values


In [8]:
from jenga.corruptions.numerical import Scaling
df = new_df()
df['A_Scaled_MAR'] = Scaling(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_Scaled_MCAR'] = Scaling(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,A_Scaled_MAR,A_Scaled_MCAR
0,0.00,c,bah,0.00,0.00
1,0.05,c,bah,0.05,0.05
2,0.10,a,foo,0.10,10.00
3,0.15,a,bah,1.50,15.00
4,0.20,b,foo,2.00,0.20
5,0.25,b,bah,2.50,0.25
6,0.30,c,bah,0.30,30.00
7,0.35,b,bah,3.50,35.00
8,0.40,a,foo,4.00,0.40
9,0.45,c,bah,0.45,0.45
